# サンプル(Classification)¶
(C)  2018-2019 Masayuki Isobe 

●はじめに

・初回実行前は、./data/pkl/配下は空にしておいてください。<br>

・./config.iniについて、実行モード・pkl自動削除モードが0になっていることを確認してください。<br>
　提出用ファイル名・特徴量数・事前評価の方法・cross validationの回数・予測確率カラム名 は、<br>
　変える必要がある場合は変えてください。

・./data/フォルダ配下に訓練用データ と 検証用データを格納願います。<BR>
　さらに、UTF-8、CRLFに変換願います。

・OneHotEncoder対象のカラム名を、"data/OneHotEncoderColumns.dat"に1行ずつに分けてUTF-8、LFで記載願います。<br>



・当ファイルを実行すると、「./output/」配下の全ファイルが削除されて新ファイルが生成されるのでご注意ください。<br>
　必要ならば事前に退避しておいてください。<br>


・実行時には、「./output/」配下のファイルを開かないでください。<br>

・「Kernel」-「Restart & Clear Output」  →  「Kernel」-「Restart & Run All」を押下して実行願います。<br>
　　初回実行時に、プロンプトにてファイル名の入力・第一カラム名の入力・性能評価指標の選択・アルゴリズムの選択を求められるのでご注意ください。<br>
    
     以下のメッセージが出力されれば終了です。
     ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
    【祝】ファイル出力が完了しました(./output/feature_nnn_yyyyy/xxxxxx.csv)♪
     ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★


●インポート

In [ ]:
#blue

# sklearn関連
from IPython.core.display import display, HTML
import pandas as pd
import numpy as np
import os
import os.path
import shutil
from IPython.core.display import display
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from time import sleep
import configparser
from sklearn.preprocessing import Imputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# アルゴリズム
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

# 性能指標
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 数値関連
import math

# パーサー
from dateutil.parser import parse

# XML
import xml.etree.ElementTree as ET
import re
import shutil

# 次元削減
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA

# HoldOut
from sklearn.model_selection import train_test_split

# k-fold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

# 描画
import matplotlib
from matplotlib import pylab as plt
from matplotlib.colors import ListedColormap
from sklearn.ensemble.partial_dependence import plot_partial_dependence

# パラメータ探索
from sklearn.model_selection import GridSearchCV

# warning無視
import warnings

# 乱数
from numpy import *


# 不均衡調整
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler,SMOTE

# pkl
import pickle

#P↓ythonでプログラムを終了させるで必要：sys.exit()
import sys

# HyperOpt
from hyperopt import fmin, tpe, hp, rand , space_eval


# ↓コメントアウトを外すと、ブレークポイントが使えるようになる。
#from IPython.core.debugger import Pdb; Pdb().set_trace()

# うざいからwarningは無視
#warnings.simplefilter('ignore')

●セルの背景色設定

In [ ]:
# セルの背景色の設定
# インポートなど：青色、関数:緑色、変数・定数宣言:黄色
# 後ろのセルから色を変える。
display(HTML("""
<script>
let cells = $(".cell");
for(let i=0;i<cells.length;i++){    
    if( $(cells[i]).text().includes("#blue") ){
        $(cells[i]).find(".CodeMirror").css("background-color","#94ffff");
    }
    else if( $(cells[i]).text().includes("#green") ){
        $(cells[i]).find(".CodeMirror").css("background-color","#efe");
    }
    else if( $(cells[i]).text().includes("#yellow") ){
        $(cells[i]).find(".CodeMirror").css("background-color","#ffff7c");
    }
    else if( $(cells[i]).text().includes("#pink") ){
        $(cells[i]).find(".CodeMirror").css("background-color","#ffc0cb");
    }
    else if( $(cells[i]).text().includes("#gray") ){
        $(cells[i]).find(".CodeMirror").css("background-color","#ededed");
    }
    else{
        $(cells[i]).find(".CodeMirror").css("background-color","#f7f7f7");
    }
}
</script>
"""))

●環境設定ファイル読み込み

In [ ]:
#blue
### 環境設定ファイル読み込み
inifile = configparser.ConfigParser()
inifile.read('./config.ini', 'UTF-8')
conf_exe_mode =  inifile.get('settings', 'exe_mode')
conf_pkl_del_mode =  inifile.get('settings', 'pkl_del_mode')
conf_input_folder_path =  inifile.get('settings', 'input_folder_path')
conf_output_folder_path =  inifile.get('settings', 'output_folder_path')
conf_xml_folder_path =  inifile.get('settings', 'xml_folder_path')
conf_pkl_folder_path =  inifile.get('settings', 'pkl_folder_path')
conf_reha_train_input_file = inifile.get('settings', 'reha_train_input_file')
conf_reha_test_input_file = inifile.get('settings', 'reha_test_input_file')
conf_submission_file = inifile.get('settings', 'submission_file')
conf_reha_one_hot_encode_column = inifile.get('settings', 'reha_one_hot_encode_column')
conf_reha_datetime_column = inifile.get('settings', 'reha_datetime_column')
conf_eval_method = int(inifile.get('settings', 'eval_method'))
conf_cv_num = int(inifile.get('settings', 'cv_num'))
conf_grid_cv_num = int(inifile.get('settings', 'grid_cv_num'))
conf_feature_num_val = int(inifile.get('settings', 'feature_num'))
conf_one_hot_encode_column_sitei = inifile.get('settings', 'one_hot_encode_column_sitei')
conf_datetime_column_sitei = inifile.get('settings', 'datetime_column_sitei')
conf_reha_first_column_name = inifile.get('settings', 'reha_first_column_name')
conf_row_num_limit_val = int(inifile.get('settings', 'row_num_limit'))
conf_unbalance_adj_mode = int(inifile.get('settings', 'unbalance_adj_mode'))
conf_col_proba = inifile.get('settings', 'col_proba')


print("conf_unbalance_adj_mode =",conf_unbalance_adj_mode)
print("conf_exe_mode =",conf_exe_mode)
print("conf_input_folder_path =",conf_input_folder_path)
print("conf_output_folder_path =",conf_output_folder_path)
print("conf_xml_folder_path =",conf_xml_folder_path)
print("conf_pkl_folder_path =",conf_pkl_folder_path)
print("conf_eval_method =",conf_eval_method)
print("conf_grid_cv_num =",conf_grid_cv_num)
print("conf_feature_num_val =",conf_feature_num_val) 
print("conf_row_num_limit_val =",conf_row_num_limit_val) 
print("conf_col_proba =",conf_col_proba) 
print("\n")

print("+++++ ↓はリハーサル時のみ使われる↓ +++++")
print("conf_reha_train_input_file =",conf_reha_train_input_file)
print("conf_reha_test_input_file =",conf_reha_test_input_file)
print("conf_submission_file =",conf_submission_file)
print("conf_reha_one_hot_encode_column =",conf_reha_one_hot_encode_column) 
print("conf_reha_datetime_column =",conf_reha_datetime_column) 
print("conf_reha_first_column_name =",conf_reha_first_column_name) 


●outputフォルダ配下の全ファイル削除

In [ ]:
#blue
#outputフォルダ配下の全ファイル削除
output_file_list=os.listdir(conf_output_folder_path)
print(output_file_list)
for ouput_file in output_file_list:
    if(os.path.isfile(conf_output_folder_path + ouput_file)):
        # ファイル削除
        os.remove(conf_output_folder_path + ouput_file)
    else:
        # フォルダ削除
        shutil.rmtree(conf_output_folder_path + ouput_file)


●変数・定数の宣言

In [ ]:
#yellow


#### 定数定義
# 実行モード:本番
EXE_MODE_PROD = "0"
# 実行モード:リハーサル
EXE_MODE_REHA = "1"
# 欠損埋めの数値
FILL_NAN_VALUE = 0
# DataFrameを表示する行数
PRINT_DF_LINENUM = 5
# 訓練用データ
FILE_TYPE_NAME_TRAIN = "訓練用データ"
# 検証用データ
FILE_TYPE_NAME_TEST = "検証用データ"
# ハイパーパラメータXML
FILE_TYPE_HYPER_PARA_NAME = "ハイパーパラメータXML"

#### 事前評価の方法
# HOLTOUT
PRE_PREDICT_HOLDOUT = "holdout"
# K_FOLD
PRE_PREDICT_K_FOLD = "k-fold"


#  TODO cv_num 事前検証用の値
CV_NUM_DEFAULT = 3
#CV_NUM_DEFAULT = 2

# 指定なし
SITEI_NASI = "0"
# 指定あり
SITEI_ARI = "1"

# 時刻型
DATE_TYPE = "datetime64[ns]"


# 型名
TYPE_INT="int"
TYPE_FLOAT="float"
TYPE_DOUBLE="double"
TYPE_BOOL="bool"
TYPE_STR="string"


# 経過時間変換の開始時刻
START_TIME = '1970-1-1 00:00:00'
# random_stateの値
RANDOM_STATE_VAL = 1
# test_size
#TEST_SIZE_VAL = 0.20
TEST_SIZE_VAL = 0.3

# 特徴量削減ステップ
SELECTOR_STEP = .05

# 評価方法の番号
K_FOLD_NUMBER = 1
H_OUT_NUMBER = 2

# 性能評価指標選択の番号
ACCURACY_NUMBER = 1
PRECISION_NUMBER = 2
RECALL_NUMBER = 3
F1_NUMBER = 4
AUC_NUMBER = 5
GO_NEXT = 99

# 不均衡調整モード
UNB_ADJ_NASI = 0
UNB_ADJ_UNDER = 1
UNB_ADJ_OVER = 2
UNB_ADJ_SMOTE = 3


# アルゴリズム名
ALG_NAME_LOGISTIC = "ロジスティック回帰"
ALG_NAME_RANDOMFOREST = "ランダムフォレスト"
ALG_NAME_MLP = "多層パーセプトロン"
ALG_NAME_XGB = "XGBoost"

# 性能評価指標・アルゴリズム選択の番号
STOP_NUMBER = 100

### 予測種別
# クラスのラベル
PREDICTTYPE_LABEL = 0
# 予測確率
PREDICTTYPE_PROBA = 1
# 予測確率対象のクラス分類の値
TARGET_PROB_CLASS_VALUE = 1
# クラス変数のカラムINDEXの値
CLASS_COL_INDEX = 1

# 描画のフォントサイズ
PLT_TITLE_FONT_SIZE=20
PLT_LABEL_FONT_SIZE=14

#### oneHotEncoder
ohe = OneHotEncoder(categorical_features=[0])

#### 性能評価選択のディクショナリ
# AUCかf1が望ましい。
perfDict = {GO_NEXT:"モデル選択へ進む",ACCURACY_NUMBER:"accuracy_score(正解率)", PRECISION_NUMBER:"precision_score(適合率)", \
    RECALL_NUMBER:"recall_score(再現率)", F1_NUMBER:"f1_score(F値)", AUC_NUMBER:"AUC", STOP_NUMBER:"中止"}

#### 性能評価ラベルのディクショナリ
perfLabelDict = {ACCURACY_NUMBER:"accuracy", PRECISION_NUMBER:"precision", \
    RECALL_NUMBER:"recall", F1_NUMBER:"f1", AUC_NUMBER:"roc_auc"}


#### モデル選択(評価方法)のディクショナリ
evalDict = {K_FOLD_NUMBER:PRE_PREDICT_K_FOLD, H_OUT_NUMBER:PRE_PREDICT_HOLDOUT}

# 特徴量上限数
feature_num_lim_val = conf_feature_num_val

#### 第一カラム名
g_first_column_name = None

#### モデルスプリット用
MODEL_SPRIT = "  "

# FLAG
FLAG_OFF = "0"
FLAG_ON = "1"


●関数の宣言(軽微なやつ)

In [ ]:
#green

####################################
##### ↓ファイル名取得関数↓
####################################
def getInputFileName(fileTypeName, backStr):
    #### ↓ループ↓
    while True:
        # ファイル名をプロンプトから入力
        input_file_name = getInputStr(fileTypeName + 'のファイル名を入力してください。' + backStr)
        
        if(fileTypeName == FILE_TYPE_NAME_TRAIN or fileTypeName == FILE_TYPE_NAME_TEST):
            directoryPath = conf_input_folder_path
        elif(fileTypeName == FILE_TYPE_HYPER_PARA_NAME):
            directoryPath = conf_xml_folder_path
        else:
            # 異常なファイルタイプ(ありえないルート)
            raise RuntimeError ("【ERROR】異常なファイルタイプです。実行を中止しました。！")

        if(os.path.isfile(directoryPath +'%s' % input_file_name)):
            return input_file_name
            #終了
        else:
            print("【ERROR】ファイルが存在しません！正しく入力してください。")
            #ループ継続
    #### ↑ループ↑


####################################
##### ↓文字列入力関数↓
####################################
def getInputStr(paramStr):
    
    #### ↓ループ↓
    while True:
        ## >>> 入力プロンプト登場
        input_str = input(paramStr + '\r\n\r\n>>>  ')
        #入力文字列が空か？
        if(input_str == None or input_str == ""):
            print("【ERROR】入力文字列が空です！")
            #ループ継続
        else:
            # 入力文字列返却
            return input_str
    #### ↑ループ↑

####################################
##### ↓DataFrame表示関数
####################################
def displayPrintDf(inputDf,printFormat,paramStr,line_num):
    if line_num == None:
        if paramStr == None or paramStr == "":
            print(printFormat)
        else:
            print(printFormat % paramStr)
        display(inputDf.head())
    else:
        if paramStr == None or paramStr == "":
            print(printFormat % (line_num))
        else:
            print(printFormat % (paramStr,line_num))
        display(inputDf.head(line_num))
    print("(行数 , 列数) = ", inputDf.shape)
    print("\r\n")

    
####################################
##### 【未使用】
##### 時刻カラム読み出し時に適用するパーサ
####################################
def myparser (dateTimeStr):
    # 欠損の場合
    if(dateTimeStr == 'nan'):
        # そのまま返す。
        # 後で呼ばれる欠損埋め処理にて平均値を入れる。
        return dateTimeStr

    # 「/」 → 「-」 に変換
    dateTimeStr = dateTimeStr.replace('/','-')
    hyphenCnt = dateTimeStr.count('-')
    colonCnt = dateTimeStr.count(':')
    # YYYY-M-D H:M ～  YYYY-MM-DD HH:MM:SS の場合
    if((hyphenCnt == 2 and (colonCnt == 1 or colonCnt == 2) ) \
        and \
        (12 <= len(dateTimeStr) and len(dateTimeStr) <= 19) ):
        #「:」の出現回数をカウント
        if(colonCnt == 2):
            # YYYY-mm-dd HH:MM:SS 形式の場合
            retDateTime = pd.datetime.strptime(dateTimeStr, '%Y-%m-%d %H:%M:%S')
        else:
            # YYYY-mm-dd HH:MM 形式の場合
            retDateTime = pd.datetime.strptime(dateTimeStr, '%Y-%m-%d %H:%M')
    else:
        # 他の 形式の場合
        # デフォルトのパーサー
        retDateTime = parse(dateTimeStr)

    return retDateTime

####################################
##### ↓DataFrameの行削除(暫定対処)↓
####################################
def delDfRow(fileTypeName, inputDf):
    retDf = inputDf
    ########
    if(fileTypeName == FILE_TYPE_NAME_TRAIN
       and
       len(inputDf) > conf_row_num_limit_val):
        
        # 訓練データの行数が上限を超えている場合
        raise ValueError ("【ERROR】訓練用データの行数が上限を超えています。確認してください。")
        
#         print("++++++++ 行数がconfigの上限を超過している！！ランダムに行を削除する。 ++++++++")
#         rowNumList = list()
#         # リストが行数の上限に達するまでループ
#         for i in range(0, len(inputDf)):
#             # 「0 ～ 行数の最大-1」のリストを生成
#             rowNumList.append(i)

#         # リストの中からランダムに選択
#         choiceList = random.choice(rowNumList, conf_row_num_limit_val, replace=False) 
        
#         # 選ばれたindexの行だけDataFrameを抽出
#         retDf = retDf.iloc[choiceList,:]
        
#         # index振り直し
#         retDf = retDf.reset_index(drop=True)

#         displayPrintDf(retDf, "行数削除直後 %s の先頭%d行：" ,fileTypeName,PRINT_DF_LINENUM)
#         print(retDf.dtypes)
    ########

    return retDf


####################################
##### ↓入力ファイルからDataFrame取得↓
####################################
def getInputDf(fileTypeName, exe_mode, fileName,  oheList, dateList, date_sitei):
    global g_first_column_name
    target_obj_dict = {}
    
    # 第一カラム名取得。初回だけ実施
    if(g_first_column_name == None):
        if(exe_mode == EXE_MODE_PROD):
            # 本番の場合 プロンプトから入力
            g_first_column_name = getInputStr('第一カラム名を入力してください')
        else:
            # リハーサルの場合 configから入力
            g_first_column_name = conf_reha_first_column_name
        
    target_obj_dict[g_first_column_name] = object
    for columnName in oheList:
        target_obj_dict[columnName] = object
            
    # OneHotEncoder対象をobject指定で読み込み 時刻指定カラムをパーサーかませて読み込み
    if(date_sitei == SITEI_ARI):
        # 時刻カラム指定あり
        inputDf = pd.read_csv(conf_input_folder_path +'%s' % fileName,\
            dtype=target_obj_dict ,parse_dates=dateList, date_parser=myparser)
    else:
        # 時刻カラム指定なし
        inputDf = pd.read_csv(conf_input_folder_path +'%s' % fileName,\
            dtype=target_obj_dict)
        
    # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # TODO
    # read_csv ハイパーパラメータの例
        # encoding=‘utf-8' or 'utf_8'  デフォルト、'shift_jis'
        # lineterminator : string, default None
        #           改行コード：  '\r'(デフォルト) or '\r\n' or '\n'
        # header=0  int (ヘッダの行番号:0始まり。デフォルトで0。)
        # quotechar クォテーションを指定するパラメータ。デフォルトは”（ダブルクォテーション）。
        #           '\'' or '"'
    # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    
    # 検証用の場合
    if(fileTypeName == FILE_TYPE_NAME_TEST):
        # ID(第1カラム)がNANの行は「未指定」に置換する。
        inputDf = inputDf.fillna({inputDf.columns.values[0]: '未指定'})

    
    # 出力
    displayPrintDf(inputDf, "csv読み込み直後 %s の先頭%d行：" ,fileTypeName,PRINT_DF_LINENUM)
    print(inputDf.dtypes)
    
    # DataFrameの行数が、configの上限を超えている場合の処理を行う。
    inputDf = delDfRow(fileTypeName, inputDf)
    
    return inputDf


####################################
#### csvファイルを読み込みんでDfを返す。
####################################
def getDfByReadCSV(fileTypeName, exe_mode, input_reha_file_name , input_ohe_clm_list, \
    input_datetime_clm_list, date_sitei):
    if(exe_mode == EXE_MODE_PROD):
        retDf = getInputDf(fileTypeName, exe_mode, getInputFileName(fileTypeName, ""), \
                           input_ohe_clm_list, input_datetime_clm_list,\
                           date_sitei)
    else:
        retDf = getInputDf(fileTypeName, exe_mode, input_reha_file_name,
                           input_ohe_clm_list, input_datetime_clm_list,\
                           date_sitei)
        
    return retDf

####################################
##### ↓One Hot Encoder関数(数値でカテゴリを指定された場合も動く)
####################################
def oneHotEncoderExe(fileTypeName, i_ohe_clm_list, inputDf, column_sitei):
    
    displayPrintDf(inputDf,"%s One Hot Encoder差し替え前の先頭%d行：" ,fileTypeName,PRINT_DF_LINENUM)

    # OneHotEncoding実施
    # NaNも一つのカテゴリーとしてダミー変数化したい場合は、引数dummy_na=Trueとする。
    if(column_sitei == SITEI_ARI):
        print("OneHotEncoding カラム指定あり")
        oheDf = pd.get_dummies(inputDf,\
            dummy_na=True,
            columns=i_ohe_clm_list)
    else:
        print("OneHotEncoding カラム指定なし")
        oheDf = pd.get_dummies(inputDf,\
            dummy_na=True)

    displayPrintDf(oheDf,"%s One Hot Encoder差し替え後の先頭%d行：" ,fileTypeName,PRINT_DF_LINENUM)

    return oheDf

####################################
##### ↓検証用データのカラム合わせ込み関数
####################################
def adjustTestDfColumn(inputTrainDf, inputTestDf):
    displayPrintDf(inputTrainDf,\
        "++++++ ↓%sの先頭%d行(カラム合わせ込み前)↓ ++++++",FILE_TYPE_NAME_TRAIN,PRINT_DF_LINENUM)
    displayPrintDf(inputTestDf,\
        "++++++ ↓%sの先頭%d行(カラム合わせ込み前)↓ ++++++",FILE_TYPE_NAME_TEST,PRINT_DF_LINENUM)

    trainColumnList = inputTrainDf.columns.values
    cols_train = set(trainColumnList)
    cols_test = set(inputTestDf.columns.values)

    # 訓練用にはあったが検証用にはないデータ項目
    diff1 = cols_train - cols_test
    #print('訓練用のみに存在する項目: %s' % diff1)
    # 検証用にはあるが訓練用になかったデータ項目
    diff2 = cols_test - cols_train
    #print('検証用のみに存在する項目: %s' % diff2)

    print('== 検証用のみに存在する項目を削除し、訓練用にのみ存在する項目を追加して初期値で埋める ==')
    # 検証用のみに存在する項目を削除
    retTestDf = inputTestDf.drop(list(diff2),\
        axis=1)

    # 訓練用データの空のDataFrameを作る。
    df_cols_train = pd.DataFrame(None,\
                             columns=trainColumnList,\
                             dtype=float)
    # df_cols_trainを、検証用データに結合(データがあるやつはそのまま)
    # 列名で紐づけて連結される
    retTestDf = pd.concat([df_cols_train, retTestDf])
    # さらにNANを初期値で埋める。
    retTestDf = retTestDf.fillna(FILL_NAN_VALUE, axis=1)

    cols_test = set(retTestDf.columns.values)
    # 訓練用にはあったが検証用にはないデータ項目
    diff1 = cols_train - cols_test
    # 検証用にはあるが訓練用になかったデータ項目
    diff2 = cols_test - cols_train
    # いずれかに差分があった場合
    if(len(diff1) > 0 or len(diff2) > 0):
        # 絶対にここに来ることはない。来たら、これより前の処理でバグがあることを意味する！！！
        raise ValueError ("【ERROR】カラム合わせこみの結果が不正です。")


    # 訓練用データのデータ項目の並び順でソートする。
    retTestDf = retTestDf.reindex(trainColumnList, axis=1)

    # この時点での検証用データ表示
    displayPrintDf(retTestDf,\
        "++++++ ↓%sの先頭%d行(カラム合わせ込み後)↓ ++++++",FILE_TYPE_NAME_TEST,PRINT_DF_LINENUM)
    return retTestDf

####################################
##### ↓ファイル出力
####################################
def fileOutput(inputDf, dirName, outFileName, lineCnt, finalFlag, i_feature_dir):
    # コンソール出力
    # 提出用CSV出力
    displayPrintDf(inputDf,\
        "++++++ ↓出力対象Dfの先頭%d行↓ ++++++", None, lineCnt)

    fileDirPath = conf_output_folder_path + i_feature_dir
    
    # 特徴量ディレクトリ
    if(os.path.exists(fileDirPath) != True):
        # ディレクトリが存在しない場合
        # ディレクトリ作成
        os.mkdir(fileDirPath)

    if(dirName != ""):
        # ディレクトリ名指定有りの場合
        if(os.path.exists(fileDirPath + dirName) != True):
            # ディレクトリが存在しない場合
            # ディレクトリ作成
            os.mkdir(fileDirPath + dirName)
        dirName = dirName + "/"

#     # 「SYLKファイルであることを確認しましたが、読み込むことができません。」
#     # を回避するため列名「ID」を"で囲む。「ID」がなければ何も変わらない。
#     inputDf = inputDf.rename(columns={'ID': '"ID"'})

    # ファイル出力
    inputDf.to_csv(fileDirPath + dirName + outFileName ,index=False, encoding="shift_jis")

    print("\n\n")
    if finalFlag == True:
        print("★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★")
        print("【祝】ファイル出力が完了しました(%s)♪" % (fileDirPath + dirName + outFileName))
        print("★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★")
    else:
        print("ファイル出力をしました(%s)。" % (fileDirPath + dirName + outFileName))

####################################
##### ↓プロンプト入力関数(入力番号を返す)
####################################
def getInputNumber(selTypeName, afterStr):
    # 1秒待つ
    sleep(1)
    #### ↓ループ↓
    while True:
        inputPerStr = selTypeName + "を入力してください。" + afterStr
        ## >>> 入力プロンプト登場
        input_item = input(inputPerStr + '\n\n>>>  ')
        #選択した評価指標が空か？
        if(input_item == None or input_item == ""):
            return None
        else:
            #入力値あり
            try:
                #int 型に変換
                return int(input_item)
            except:
                print("入力値が不正です。再入力してください。\r\n" , e)
                sleep(1)
    #### ↑ループ↑
        
####################################
##### ↓プロンプト入力関数(入力番号を返す(ディクショナリ))
####################################
def getInputNumberDic(selTypeName,inputDict):
    # 1秒待つ
    sleep(1)
    #### ↓ループ↓
    while True:
        inputPerStr = selTypeName + "を選択してください。" + str(inputDict)
        ## >>> 入力プロンプト登場
        input_item = input(inputPerStr + '\n\n>>>  ')
        #選択した評価指標が空か？
        if(input_item == None or input_item == ""):
            print("【ERROR】入力してください！")
            #ループ継続
        else:
            #int 型に変換
            input_item_num = int(input_item)
            
            value_item = inputDict.get(input_item_num)
            if(value_item == None or value_item == ""):
                print("【ERROR】範囲内の値を入力してください！")
                #ループ継続
            else:
                print("入力されたのは [%s:%s] です" % (input_item_num,value_item))
                return input_item_num
    #### ↑ループ↑

####################################
##### ↓外部ファイルを読み込んで値をリストで返却する関数
####################################
def getFileValueList(sitei_mode, filePath, fileName):
    value_list = list()
    
    if(sitei_mode == SITEI_ARI):
        # 指定ありの場合 ファイルを読み込んでリストに詰める
        value_data = open(filePath + fileName, "r", encoding="utf-8")
        # TODO 文字コード指定
        # Shift-JIS の場合は、encoding="shift_jis"
        for value_data_line in value_data:
            value_data_line = value_data_line.replace('\n','').replace('\r','')
            value_list.append(value_data_line)
            

    print("★" + fileName + "から読み込んだカラム=",value_list)
    sleep(1)
    return value_list

####################################
##### ↓設定値決定
####################################
def getSettingValue(confSettingValue,prodSettingValue,exe_mode):
    if(exe_mode == EXE_MODE_PROD):
        retSettingValue = prodSettingValue
    else:
        retSettingValue = confSettingValue 
        
    return retSettingValue

####################################
##### 【未使用】
##### 経過時間の計算
##### ↓日付時分秒のデータを経過時間に置き換える。
####################################
def getConvTimeDf(fileTypeName, inputDf):
    # 変換前
    displayPrintDf(inputDf,fileTypeName + " 時刻変換前inputDfの先頭%d行：" ,None,PRINT_DF_LINENUM)

#     print("スケールが大きいので対数を取る") 
#     dateTimeDf['Outlet_Establishment_Year'] = math.log(dateTimeDf['Outlet_Establishment_Year'])
#     displayPrintDf(dateTimeDf,fileTypeName + " dateTimeDfの先頭%d行：" ,None,PRINT_DF_LINENUM)

    column_list = inputDf.columns
    for columnName in column_list:
        # 時刻型の場合
        if(inputDf[columnName].dtype == DATE_TYPE):
            print("時刻型のカラム：",columnName)
            dateTimeDf = inputDf[columnName]
            # UNIX開始時刻からの経過時間に変換する。
            timediff = dateTimeDf - pd.to_datetime(START_TIME,format='%Y%m%d %H:%M:%S')
            dateTimeDf = timediff/np.timedelta64(1, 's')
            #変換した値に置換する
            inputDf[columnName] = dateTimeDf
    
    # 変換後
    displayPrintDf(inputDf,fileTypeName + " 時刻変換後inputDfの先頭%d行：" ,None,PRINT_DF_LINENUM)

    return inputDf

####################################
##### 次元削減(特徴量選択) を使用する
##### ↓特徴量選択(RandomForestClassifier)
####################################
def getSelectorFeature(fileTypeName,inputFeatureDf,y_Df,feature_num):
    # 特徴量選択前
    displayPrintDf(inputFeatureDf,fileTypeName + " 特徴量選択前Dfの先頭%d行：" ,None,PRINT_DF_LINENUM)

    # 特徴量因子の重要度を推定する分類器をRandomForestClassifierに設定
    # 最終的に残す特徴量をconfig.iniに定義している
    # 0未満 の場合はNoneに設定する。
    # 1回のstepで削除する次元数は5%ずつとする
    
    print("特徴量選択数 = ",feature_num)
    if(feature_num < 0):
        # デフォルト値
        print("n_features_to_select = None に設定")
        feature_num = None
    
    # 特徴量選択
    # step:特徴量削除の速度。一度の再帰処理により指定ステップ分の特徴量が消滅する。
    selector = RFE(estimator=RandomForestClassifier(random_state=RANDOM_STATE_VAL),
                   n_features_to_select=feature_num,
                   step=SELECTOR_STEP)
    selector.fit(inputFeatureDf,y_Df.values.ravel())

    # 削除対象配列(Falseが削除対象)
    selFlagArray = selector.support_
    
    # 現状のカラム名を取得
    columnIndex = inputFeatureDf.columns
    
    # 削除対象カラム名リスト
    drop_column_list = list()
    # 削除対象配列 でループ(i=0,1,2,・・・)
    print(len(selFlagArray))
    for i in range(len(selFlagArray)):
        # Falseの場合に、削除対象リストに追加
        if(selFlagArray[i] == False):
            drop_column_list.append(columnIndex[i])

    # 不要な列削除
    #print("削除対象カラム = " , drop_column_list)
    inputFeatureNewDf = inputFeatureDf.drop(drop_column_list, axis=1)
    
    # 特徴量選択後
    displayPrintDf(inputFeatureNewDf,fileTypeName + " 特徴量選択後Dfの先頭%d行：" ,None,PRINT_DF_LINENUM)

    return inputFeatureNewDf


####################################
##### ↓TRIM関数↓
####################################
def strTrim(inputStr):
    trimStr = inputStr.strip()
    return trimStr


####################################
##### ↓XML読み込み↓
####################################
def xmlRead(fileName):
    hyperOptFlag = False
    xmlDict = dict()
    # paramsタグ
    # xmlファイルの読み込み
    tree = ET.parse(conf_xml_folder_path + fileName)
    root = tree.getroot()
    for child1 in root:
        c1Text = strTrim(child1.text)
        if(c1Text != None and c1Text != ""):
            # algorithm
            if(child1.tag in xmlDict):
                raise ValueError ("【ERROR】タグが重複しています：" + child1.tag)
            algoStr = c1Text
        for child2 in child1:

            # パラメータ名
            if(child2.tag in xmlDict):
                raise ValueError ("【ERROR】タグが重複しています：" + child2.tag)

            xmlDict[child2.tag] = []
            rangeMin = 0
            rangeMax = 0
            rangeIncre = 0
            hpChoiceArray = []
            lowVal = None

            for child3 in child2:
                # 型、値
                paramArray = xmlDict[child2.tag]
                c3Text = strTrim(child3.text)
                if(c3Text == None or c3Text == ""):
                    raise ValueError ("【ERROR】値が空です：" + child3.tag)
                else:
                    if(child3.tag == "type"):
                        valueType = c3Text
                    ############################################################################
                    # ここからはGridSearchCV用。
                    ############################################################################
                    elif(re.match('value*', child3.tag)):
                        if(valueType == TYPE_INT):
                            # int型の場合
                            paramVal = int(c3Text)
                        elif(valueType == TYPE_FLOAT):
                            # float型の場合
                            paramVal = float(c3Text)
                        elif(valueType == TYPE_DOUBLE):
                            # double型の場合
                            paramVal = double(c3Text)
                        elif(valueType == TYPE_BOOL):
                            # bool型の場合
                            # まずは文字列で読みだしてからboolに変換する。
                            paramVal = str(c3Text)
                            if(paramVal == "True"):
                                paramVal = True
                            elif(paramVal == "False"):
                                paramVal = False
                            else:
                                raise ValueError ("【ERROR】値が不正です：" + child3.tag)
                        else:
                            # string型の場合
                            paramVal = str(c3Text)

                        # 設定
                        paramArray.append(paramVal)
                        xmlDict[child2.tag] = paramArray
                    elif(re.match('rangeMin', child3.tag)):
                        if(valueType == TYPE_INT):
                            # int型の場合
                            rangeMin = int(c3Text)
                        elif(valueType == TYPE_FLOAT):
                            # float型の場合
                            rangeMin = float(c3Text)
                        elif(valueType == TYPE_DOUBLE):
                            # double型の場合
                            rangeMin = double(c3Text)
                    elif(re.match('rangeMax', child3.tag)):
                        if(valueType == TYPE_INT):
                            # int型の場合
                            rangeMax = int(c3Text)
                        elif(valueType == TYPE_FLOAT):
                            # float型の場合
                            rangeMax = float(c3Text)
                        elif(valueType == TYPE_DOUBLE):
                            # double型の場合
                            rangeMax = double(c3Text)
                    elif(re.match('rangeIncre', child3.tag)):
                        if(valueType == TYPE_INT):
                            # int型の場合
                            rangeIncre = int(c3Text)
                        elif(valueType == TYPE_FLOAT):
                            # float型の場合
                            rangeIncre = float(c3Text)
                        elif(valueType == TYPE_DOUBLE):
                            # double型の場合
                            rangeIncre = double(c3Text)
                        # xmlDictに設定
                        xmlDict[child2.tag] = range(rangeMin, rangeMax, rangeIncre)
                    ############################################################################
                    # ここからはHyperOpt用。
                    ############################################################################
                    elif(re.match('hp.*', child3.tag)):
                        # HyperOpt
                        hyperOptFlag = True
                        xmlDict, lowVal , hpChoiceArray \
                            = xmlReadHyperOpt(xmlDict, valueType ,lowVal, hpChoiceArray, child2.tag, child3.tag, c3Text)
                    ############################################################################
                    # その他のタグ
                    ############################################################################
                    else:
                        raise ValueError ("【ERROR】タグが不正です：" + child3.tag)

    return algoStr , xmlDict , hyperOptFlag

####################################
##### ★バグがあるので未使用★
##### ↓XML読み込みサブロジック(HyperOpt)↓
####################################
def xmlReadHyperOpt(inXmlDict, inValueType ,inLowVal, inChoiceArray, inC2tag, inC3tag, inC3Text):
    xmlDict = inXmlDict
    lowVal = inLowVal
    choiceArray = inChoiceArray
    
    if(re.match('hp.uniformLow', inC3tag) or re.match('hp.loguniformLow', inC3tag)):
        if(inValueType == TYPE_INT):
            # int型の場合
            lowVal = int(inC3Text)
        elif(inValueType == TYPE_FLOAT):
            # float型の場合
            lowVal = float(inC3Text)
        elif(inValueType == TYPE_DOUBLE):
            # double型の場合
            lowVal = double(inC3Text)
    elif(re.match('hp.uniformHigh', inC3tag)):
        if(inValueType == TYPE_INT):
            # int型の場合
            # xmlDictに設定
            xmlDict[inC2tag] = hp.uniform(inC2tag, int(inLowVal), int(inC3Text))
        elif(inValueType == TYPE_FLOAT):
            # float型の場合
            # xmlDictに設定
            xmlDict[inC2tag] = hp.uniform(inC2tag, float(inLowVal), float(inC3Text))
        elif(inValueType == TYPE_DOUBLE):
            # double型の場合
            # xmlDictに設定
            xmlDict[inC2tag] = hp.uniform(inC2tag, double(inLowVal), double(inC3Text))
    elif(re.match('hp.loguniformHigh', inC3tag)):
        if(inValueType == TYPE_INT):
            # int型の場合
            xmlDict[inC2tag] = hp.loguniform(inC2tag, np.log(int(inLowVal)), np.log(int(inC3Text)))
        elif(inValueType == TYPE_FLOAT):
            # float型の場合
            xmlDict[inC2tag] = hp.loguniform(inC2tag, np.log(float(inLowVal)), np.log(float(inC3Text)))
        elif(inValueType == TYPE_DOUBLE):
            # double型の場合
            xmlDict[inC2tag] = hp.loguniform(inC2tag, np.log(double(inLowVal)), np.log(double(inC3Text)))
    elif(re.match('hp.randint', inC3tag)):
        # xmlDictに設定
        xmlDict[inC2tag] = hp.randint(inC2tag, int(inC3Text))
    elif(re.match('hp.choice*', inC3tag)):
        if(inValueType == TYPE_INT):
            # int型の場合
            paramVal = int(inC3Text)
        elif(inValueType == TYPE_FLOAT):
            # float型の場合
            paramVal = float(inC3Text)
        elif(inValueType == TYPE_DOUBLE):
            # double型の場合
            paramVal = double(inC3Text)
        elif(inValueType == TYPE_BOOL):
            # bool型の場合
            # まずは文字列で読みだしてからboolに変換する。
            paramVal = str(inC3Text)
            if(paramVal == "True"):
                paramVal = True
            elif(paramVal == "False"):
                paramVal = False
            else:
                raise ValueError ("【ERROR】値が不正です：" + inC3tag)
        else:
            # string型の場合
            paramVal = str(inC3Text)
        choiceArray.append(paramVal)
        # ラストのタグの場合にxmlDictに設定
        if(inC3tag == "hp.choiceLast"):
            xmlDict[inC2tag] = hp.choice(inC2tag, choiceArray)
        
    return xmlDict, lowVal, choiceArray

####################################
##### ↓ランクDFからディクショナリを生成する関数
####################################
def createDictFromRankDf(scoringName, nameArray, createDict, pipelines):
    # 順位でソートしたアルゴリズムの配列
    for elem in nameArray:
        # パイプラインのループ
        for pipe_name, pipeline in pipelines.items():
            # 該当のアルゴリズムを検索
            if(elem == pipe_name):
                createDict[scoringName + MODEL_SPRIT + pipe_name] = pipeline
                break

        
    return createDict

# ####################################
# ##### 【未使用】
# ##### ↓不均衡調整を行う関数
# ####################################
# def unbalanceAdj(adjMode, X_inputDf, y_inputDf):
#     print("=============================== unbalanceAdj START ===============================")
#     print("【第1引数】adjMode：" ,adjMode)


#     print("不均衡調整実施前：",Counter(y_inputDf.as_matrix().ravel()))
    
#     if(adjMode == UNB_ADJ_NASI):
#         # 不均衡調整モード＝なし の場合
#         ret_XDf = X_inputDf
#         ret_yDf = y_inputDf
#         print("不均衡調整実施なし")
#     elif(adjMode == UNB_ADJ_UNDER):
#         # 不均衡調整モード＝アンダーサンプリング の場合
#         rus = RandomUnderSampler(random_state=RANDOM_STATE_VAL)
#         X_under, y_under = rus.fit_sample(X_inputDf, y_inputDf)
#         print("不均衡調整実施後(アンダーサンプリング)：",Counter(y_under))
#         ret_XDf = pd.DataFrame(X_under, columns= X_inputDf.columns.values)
#         ret_yDf = pd.DataFrame(y_under, columns= y_inputDf.columns.values)
#     elif(adjMode == UNB_ADJ_OVER):
#         # 不均衡調整モード＝オーバーサンプリング の場合
#         ros = RandomOverSampler(random_state=RANDOM_STATE_VAL)
#         X_over, y_over = ros.fit_sample(X_inputDf, y_inputDf)
#         print("不均衡調整実施後(オーバーサンプリング)：",Counter(y_over))
#         ret_XDf = pd.DataFrame(X_over, columns= X_inputDf.columns.values)
#         ret_yDf = pd.DataFrame(y_over, columns= y_inputDf.columns.values)
#     elif(adjMode == UNB_ADJ_SMOTE):
#         # 不均衡調整モード＝SMOTE の場合
#         smt = SMOTE(random_state=RANDOM_STATE_VAL)
#         X_smt, y_smt = smt.fit_sample(X_inputDf, y_inputDf)
#         print("不均衡調整実施後(SMOTE)：",Counter(y_smt))
#         ret_XDf = pd.DataFrame(X_smt, columns= X_inputDf.columns.values)
#         ret_yDf = pd.DataFrame(y_smt, columns= y_inputDf.columns.values)
#     else:
#         # 不均衡調整モード＝その他 の場合
#         raise ValueError ("【ERROR】不均衡調整モードの値が不正です。：" + adjMode)

#     ####
#     displayPrintDf(ret_XDf,\
#         "++++++ ↓%sの先頭%d行(不均衡調整後X)↓ ++++++",FILE_TYPE_NAME_TRAIN,PRINT_DF_LINENUM)
#     print(ret_XDf.dtypes)

#     displayPrintDf(ret_yDf,\
#         "++++++ ↓%sの先頭%d行(不均衡調整後y)↓ ++++++",FILE_TYPE_NAME_TRAIN,PRINT_DF_LINENUM)
#     print(ret_yDf.dtypes)
    
#     return ret_XDf, ret_yDf


●関数の宣言(重いやつ)

In [ ]:
#pink

####################################
##### ↓ディクショナリをキー入力させる関数
####################################
def getInputNumberKeyDict(inputDict, dispStr):
    input_pipe_key_Dict=dict()
    for index,key in enumerate(inputDict):
        input_pipe_key_Dict[index+1] = key
    # 末尾に中止を追加
    input_pipe_key_Dict[STOP_NUMBER] = "中止"
    
    # アルゴリズム選択
    algNumber = getInputNumberDic(dispStr, input_pipe_key_Dict)
    if (algNumber == STOP_NUMBER):
        raise RuntimeError ("【ERROR】実行を中止しました。！")
        
    retVal = inputDict.get(input_pipe_key_Dict[algNumber])
    keyStr = input_pipe_key_Dict[algNumber]

    return keyStr, retVal
        
####################################
##### ↓ハイパーパラメータをXMLから取得する関数
####################################
def getHyperPara(algorithm, inputFileName):
    readAlgo , readParaDict, hyperOptFlag = xmlRead(inputFileName)
    
    if(algorithm != readAlgo):
        raise ValueError ("【ERROR】XMLのアルゴリズムが不正です。" + readAlgo)
    
    print("++++++++++++++++++++++++++++++++++++")
    print("XMLのアルゴリズム = ",readAlgo)
    print("XMLのハイパーパラメータ = ",readParaDict)
#     print("HyperOptFlag = ",hyperOptFlag)
    print("++++++++++++++++++++++++++++++++++++")
    return readParaDict, hyperOptFlag

####################################
##### ↓【最重要】ベストなモデルを選択して返す関数
# 引数 訓練用の特徴量,訓練用の正解
####################################
def getBestModelClf(X_input, y_input, i_feature_num, i_feature_dir, i_cv_num):
    
    # デフォルトのモデル
    prePredict, scoring , algorithm , defaultClfPipe, defaultRankDf, feature_num = \
        getBestModelDefaultClfLogic(X_input, y_input, i_feature_num, i_feature_dir)
    
    ## モデル最終決定用のディクショナリ
    model_Dict=dict()

    ############# ↓交差検証の場合↓ #############
    if(conf_eval_method == K_FOLD_NUMBER):
        # 交差検証の場合にのみパラメータチューニングを行う。
        model_Dict[scoring + MODEL_SPRIT + algorithm] = defaultClfPipe
        inputRankDf = defaultRankDf

        ###### ↓wheileループ(パラメータチューニング)↓ ######
        tuneNo = 0
        while True:
            inputFileName = getInputFileName(FILE_TYPE_HYPER_PARA_NAME,
                                             "\r\nこれ以上行わない場合は「c」を入力してください。")

            if(inputFileName == 'c'):
                # ループを抜ける。
                break
            else:
                print("=== パラメータチューニングを実行します。===")
                tuneNo = tuneNo + 1

                try:
                    # ハイパーパラメータチューニングを実施
                    model_Dict, inputRankDf = getBestModelTuneClfLogic(prePredict, X_input,
                                y_input, scoring , algorithm, defaultClfPipe, 
                                inputRankDf, tuneNo, inputFileName, model_Dict, feature_num, i_feature_dir,
                                i_cv_num)
                except ValueError as e:
                    print("例外発生！：\r\n" , e)
                    sleep(1)
        ###### ↑wheileループ(パラメータチューニング)↑ ######
    ############# ↑交差検証の場合↑ #############
    
    ## モデル最終決定
    if(len(model_Dict) > 1):
        # パラメータチューニングした場合
        print("パラメータチューニングあり")
        key, retPipe = getInputNumberKeyDict(model_Dict, "モデル")
    else:
        # パラメータチューニングしていない場合
        print("パラメータチューニングなし")
        retPipe = defaultClfPipe

    return retPipe


# ####################################
# ##### ★未使用★
# ##### ↓HyperOptCVのサブロジック
# ####################################
# def hyperOptLogic(pipeline, paramDic, X_train, y_train, X_test, y_test, scoringName):
#     pipeline.fit(X_train,
#                 y_train)
#     return -f1_score(y_test, pipeline.predict(X_test), pos_label=TARGET_PROB_CLASS_VALUE)

# ####################################
# ##### ★未使用★
# ##### ↓HyperOptCVExeを実行して
# #####   後のGridSearchCVに渡すパラメータを生成する。
# ####################################
# def hyperOptCVExe(pipeline, paramDic, X_input, y_input, scoringName):
    
#     # 元の訓練データを訓練用と検証用に分ける
#     X_train, X_test, y_train, y_test = train_test_split(X_input,     # 特徴量
#                                              y_input,                # 正解データ
#                                              test_size=TEST_SIZE_VAL ,
#                                              random_state=RANDOM_STATE_VAL)
#     y_train_ravel = y_train.values.ravel()
#     y_test_ravel = y_test.values.ravel()

#     convParamDic = fmin(hyperOptLogic(pipeline, paramDic, X_train, y_train_ravel, X_test, y_test_ravel, scoringName), \
#                         paramDic, algo=tpe.suggest, max_evals=100)
#     print("best estimate parameters = ", convParamDic)

#     # GridSearchCVに渡すパラメータディクショナリを返す
#     return convParamDic

####################################
##### ↓【最重要】ベストなモデルを選択して返す関数(Tune)
# 引数 訓練用の特徴量,訓練用の正解
####################################
def getBestModelTuneClfLogic(prePredict, X_input, y_input, scoringName , algorithm, clfPipe, inputRankDf, 
    tuneNo, inputFileName, inputModelDct, i_feature_num, i_feature_dir, i_cv_num):
    print("=============================== getBestModelTuneClfLogic START ===============================")
    print("【第1引数】prePredict：" ,prePredict)
    displayPrintDf(X_input,"【第2引数】X_inputの先頭%d行：" ,None,PRINT_DF_LINENUM)
    displayPrintDf(y_input,"【第3引数】y_inputの先頭%d行：" ,None,PRINT_DF_LINENUM)
    print("【第4引数】scoringName：" ,scoringName)
    print("【第5引数】algorithm：" ,algorithm)
    print("【第6引数】clfPipe：" ,clfPipe)
    displayPrintDf(inputRankDf,"【第7引数】inputRankDfの先頭%d行：" ,None,PRINT_DF_LINENUM)
    print("【第8引数】tuneNo：" ,str(tuneNo))
    print("【第9引数】inputFileName：" ,inputFileName)
    print("【第10引数】inputModelDct：" ,inputModelDct)
    print("【第11引数】i_feature_num：" ,i_feature_num)
    print("【第12引数】i_feature_dir：" ,i_feature_dir)
    print("【第13引数】i_cv_num：" ,i_cv_num)

    
    # warning消し
    #.as_matrix will be removed in a future version. Use .values instead.
    #y_input_ravel = y_input.as_matrix().ravel()
    y_input_ravel = y_input.values.ravel()
    
    g_param1 , hyperOptFlag = getHyperPara(algorithm, inputFileName)
    
#     if(hyperOptFlag):
#         # TODO TypeError: 'numpy.float64' object is not callable が解決するまで封印
#         # HyperOptCVを実行
#         # GridSearchCVにつなげるためg_param1のキーを変換する。
#         g_param1 = hyperOptCVExe(clfPipe,
#             g_param1,
#             X_input, y_input, scoringName)
#         #####################################
    
    # n_jobs=-1とすると、CPUコア数の数だけ並列化される
    estVal = GridSearchCV(estimator = clfPipe,
                            param_grid = g_param1,
                            n_jobs=-1,
                            scoring=scoringName,
                            cv=conf_grid_cv_num,
                            return_train_score=False)
    
    # パイプラインディクショナリに当チューニングを追加
    pipelineDct = {
        algorithm + "_tune" + str(tuneNo) : estVal
    }

    print("############################################# k-fold #############################################")
    ### k-foldの結果を取得
    k_fold_Df = \
        getKFoldResult(scoringName, pipelineDct, X_input, y_input_ravel, i_cv_num, True)
    
    
    # 精度の大きい順に順位付けしてソートし直す。
    k_fold_Df = getRankDf(k_fold_Df, '精度', 'algorithm', True)

    #    0:algorithm  1:精度       2:順位
    # → 0:順位       1:algorithm  2:精度
    k_fold_Df = (k_fold_Df.iloc[:,[2]]).join(k_fold_Df.iloc[:,[0,1]])

    # 【要改善】デフォルトDfをくっつけて
    # 精度の大きい順に順位付けしてソートし直す。
    k_fold_Df = pd.concat( [inputRankDf, k_fold_Df] )
    k_fold_Df = getRankDf(k_fold_Df, '精度', 'algorithm', False)

    # ファイル出力(フォルダ名には拡張子の「.xml」は含まない)
    outDir = "tune" + str(tuneNo) + "_" + inputFileName[:-4]
    fileOutput(k_fold_Df, outDir ,
               str(i_feature_num) + "_" + PRE_PREDICT_K_FOLD + 
               "_perf_" + scoringName + "_tune" + str(tuneNo) + ".csv",
               len(k_fold_Df) , False, i_feature_dir)
    # XMLファイルをコピー
    shutil.copyfile(conf_xml_folder_path + inputFileName,
                    conf_output_folder_path + i_feature_dir  + outDir + "/" + inputFileName)

    # パイプラインディクショナリに載せ替え
    for pipe_name, pipeline in inputModelDct.items():
        # pipe_nameをスペースの前後で分解
        scoring , modelName = pipe_name.split()
        pipelineDct[modelName] = pipeline
    
    # アルゴリズムの順位ごとにパイプラインをディクショナリで保持
    # warning消し
    #.as_matrix will be removed in a future version. Use .values instead.
#     modelDict = createDictFromRankDf(scoringName, k_fold_Df['algorithm'].as_matrix().ravel(),
#         dict(), pipelineDct)
    modelDict = createDictFromRankDf(scoringName, k_fold_Df['algorithm'].values.ravel(),
        dict(), pipelineDct)

    ## k-fold 描画
    rankDfKfoldDict = dict()
    rankDfKfoldDict[scoringName]=k_fold_Df
    plotPerf(rankDfKfoldDict, K_FOLD_NUMBER)


    return modelDict , k_fold_Df

####################################
##### ↓【最重要】ベストなモデルを選択して返す関数(デフォルト)
# 引数 訓練用の特徴量,訓練用の正解
####################################
def getBestModelDefaultClfLogic(X_input, y_input, i_feature_num, i_feature_dir):
    print("=============================== getBestModelClf START ===============================")
    displayPrintDf(X_input,"【第1引数】X_inputの先頭%d行：" ,None,PRINT_DF_LINENUM)
    displayPrintDf(y_input,"【第2引数】y_inputの先頭%d行：" ,None,PRINT_DF_LINENUM)
    print("【第3引数】i_feature_num：" ,i_feature_num)
    print("【第4引数】i_feature_dir：" ,i_feature_dir)
    #######################

    # 元の訓練データを訓練用と検証用に分ける(HoldOut用)
    inputTrainFeatureDf, inputTestFeatureDf, y_train, y_test = train_test_split(X_input,     # 特徴量
                                                 y_input,    # 正解データ
                                                 test_size=TEST_SIZE_VAL ,
                                                 random_state=RANDOM_STATE_VAL)

    print("===== スプリット後 =====")
    displayPrintDf(inputTrainFeatureDf,"inputTrainFeatureDfの先頭%d行：" ,None,PRINT_DF_LINENUM)
    displayPrintDf(y_train,"y_trainの先頭%d行：" ,None,PRINT_DF_LINENUM)
    displayPrintDf(inputTestFeatureDf,"inputTestFeatureDfの先頭%d行：" ,None,PRINT_DF_LINENUM)
    displayPrintDf(y_test,"y_testの先頭%d行：" ,None,PRINT_DF_LINENUM)

    
    
    # 特徴量数の調整
    X_clomns_num = len(X_input.columns)
    if(i_feature_num <= X_clomns_num):
        # DataFrameの特徴量の数がコンフィグ/コマンドの設定値以上の場合
        # コンフィグ/コマンドの設定値を採用する
        feature_num_val_adj = i_feature_num
    else:
        # DataFrameの特徴量の数がコンフィグ/コマンドの設定値よりも少ない場合
        # DataFrameの特徴量の数を採用する
        feature_num_val_adj = X_clomns_num
        
    print("++++++++++++++++++++++++++++++++++++++++++++++++")
    print("調整後の特徴量の上限数 = ", feature_num_val_adj)
    print("++++++++++++++++++++++++++++++++++++++++++++++++")

    # warning消し
    #.as_matrix will be removed in a future version. Use .values instead.
    #y_train_ravel = y_train.as_matrix().ravel()
    #y_test_ravel = y_test.as_matrix().ravel()
    #y_input_ravel = y_input.as_matrix().ravel()
    y_train_ravel = y_train.values.ravel()
    y_test_ravel = y_test.values.ravel()
    y_input_ravel = y_input.values.ravel()
    
    

    clf_pipe_Hout_Dict=dict()
    clf_pipe_Kfold_Dict=dict()
    rankDfKfoldDict = dict()
    rankDfHoutDict = dict()

    #####################################################
    ################## ↓whileループ↓ ##################
    #####################################################
    selPerfFlag = False
    while True:
        perf_number = getInputNumberDic("性能評価指標",perfDict)

        # 'アルゴリズム選択へ進む'の場合
        if perf_number == GO_NEXT:
            if selPerfFlag == True:
                # 性能評価指標が選択済みの場合
                print("性能評価指標選択を終了します。")
                break;
            else:
                # 性能評価指標が未選択の場合
                print("【ERROR】性能評価指標が未選択です。")
                continue
        elif (perf_number == STOP_NUMBER):
            raise RuntimeError ("【ERROR】実行を中止しました。！")

        # 選択済みフラグON
        selPerfFlag = True

        # 性能評価ラベルの取得
        perf_label = perfLabelDict.get(perf_number)
        
        # アルゴリズム取得
        pipelines = getPipelines(feature_num_val_adj)

        ############# ↓交差検証の場合↓ #############
        if(conf_eval_method == K_FOLD_NUMBER):
            print("############################################# k-fold #############################################")
            ### k-foldの結果を取得(この時点でのcvは少ない数値で行う。)
            k_fold_Df = \
                getKFoldResult(perf_label, pipelines, X_input, y_input_ravel, CV_NUM_DEFAULT, False)

            # 精度の大きい順に順位付けしてソートし直す。
            k_fold_Df = getRankDf(k_fold_Df, '精度', 'algorithm', True)

            #    0:algorithm  1:精度       2:順位
            # → 0:順位       1:algorithm  2:精度
            k_fold_Df = (k_fold_Df.iloc[:,[2]]).join(k_fold_Df.iloc[:,[0,1]])
            fileOutput(k_fold_Df, "default" ,
                       str(i_feature_num) + "_" + PRE_PREDICT_K_FOLD + "_perf_"+ perf_label 
                        +".csv", len(pipelines), False, i_feature_dir)

            # アルゴリズムの順位ごとにパイプラインをディクショナリで保持
    # warning消し
    #.as_matrix will be removed in a future version. Use .values instead.
#             clf_pipe_Kfold_Dict = createDictFromRankDf(perf_label, k_fold_Df['algorithm'].as_matrix().ravel(),
#                 clf_pipe_Kfold_Dict, pipelines)
            clf_pipe_Kfold_Dict = createDictFromRankDf(perf_label, k_fold_Df['algorithm'].values.ravel(),
                clf_pipe_Kfold_Dict, pipelines)

            ## k-fold 描画
            rankDfKfoldDict[perf_label]=k_fold_Df
            plotPerf(rankDfKfoldDict, K_FOLD_NUMBER)
            
        ############# ↓Hold Outの場合↓ #############
        else:

            print("############################################# HoldOut #############################################")
            ### HoldOutの結果を取得
            rankDf = \
                getHoldOutResult(perf_number, perf_label, pipelines, \
                    inputTrainFeatureDf, y_train_ravel, inputTestFeatureDf, y_test_ravel)
        
            # testの値が大きい順に順位付けしてソートし直す。
            rankDf = getRankDf(rankDf, rankDf.columns.values[1], 'algorithm', True)


            #    0:algorithm  1:train-評価  2:test-評価  3:順位
            # → 0:順位       1:algorithm   2:test-評価  3:train-評価    
            rankDf = (rankDf.iloc[:,[3]]).join((rankDf.iloc[:,[0]])).join((rankDf.iloc[:,[2]])) \
                .join(rankDf.iloc[:,[1]])

            fileOutput(rankDf, "default", 
                       str(i_feature_num) + "_" + PRE_PREDICT_HOLDOUT+ "_perf_"+ perf_label +
                       ".csv", len(pipelines), False, i_feature_dir)
        
            # アルゴリズムの順位ごとにパイプラインをディクショナリで保持
            # warning消し
            #.as_matrix will be removed in a future version. Use .values instead.
#             clf_pipe_Hout_Dict = createDictFromRankDf(perf_label, rankDf['algorithm'].as_matrix().ravel(),
#                 clf_pipe_Hout_Dict, pipelines)
            clf_pipe_Hout_Dict = createDictFromRankDf(perf_label, rankDf['algorithm'].values.ravel(),
                clf_pipe_Hout_Dict, pipelines)

    
            ## HoldOut 描画
            rankDfHoutDict[perf_label]=rankDf
            plotPerf(rankDfHoutDict, H_OUT_NUMBER)
    #####################################################
    ################## ↑whileループ↑ ##################
    #####################################################

    if(conf_eval_method == K_FOLD_NUMBER):
        # k-foldの場合
        modelDict = clf_pipe_Kfold_Dict
        prePredict = PRE_PREDICT_K_FOLD
        selRankDict = rankDfKfoldDict
    else:
        # HoldOutの場合
        modelDict = clf_pipe_Hout_Dict
        prePredict = PRE_PREDICT_HOLDOUT
        selRankDict = rankDfHoutDict

    ##################
    ### アルゴリズムをコンソールから選択して学習モデル返却
    # パイプラインのキーをディクショナリで番号で管理
    keyStr, selClfPipe = getInputNumberKeyDict(modelDict, "モデル")
    scoring , algorithmVal = keyStr.split()
    
    selRankDf = selRankDict[scoring]
    # 指定したアルゴリズムの行だけ抜き出す(where句でhitしなかった行はnaなのでdropする)
    selRankDf = selRankDf.where(selRankDf['algorithm'] == algorithmVal).dropna()
    
    return prePredict, scoring , algorithmVal , selClfPipe , selRankDf, feature_num_val_adj


####################################
##### ↓パイプライン取得
####################################
def getPipelines(i_feature_num_val_adj):
    ### アルゴリズムのパイプライン ###
    # set pipelines for different algorithms
    pipelines = {
# 欠番
#             # :K近傍法
#             'knn(K近傍法)':
#                 Pipeline([('scl',StandardScaler()),
#                           ('reduct', PCA(n_components=i_feature_num_val_adj, random_state=RANDOM_STATE_VAL)),
#                           ('est',KNeighborsClassifier())]),

        # 1:ロジスティック回帰
        ALG_NAME_LOGISTIC:
            Pipeline([('scl',StandardScaler()),
                      # TODO 必要に応じて次元圧縮復活
                      #('reduct', PCA(n_components=i_feature_num_val_adj,random_state=RANDOM_STATE_VAL)),
                      # 「log_tune_TPOT.xml」 に準拠
                      ('est',LogisticRegression(C=15.0, random_state=RANDOM_STATE_VAL))]),
                      # デフォルト
                      #('est',LogisticRegression(random_state=RANDOM_STATE_VAL))]),

    # ×欠番 (大量データだとやたら計算が遅い)
    #     # :SVC(kernel='rbf' RBFカーネルのパラメータ: γは, 以下のRBFカーネルの式の中で現れます)
    #     'rsvc(SVC)':
    #         Pipeline([('scl',StandardScaler()),
    #                   ('reduct', PCA(n_components=i_feature_num_val_adj, random_state=RANDOM_STATE_VAL)),
    #                   ('est',SVC(C=1.0,kernel='rbf',class_weight='balanced',random_state=RANDOM_STATE_VAL, probability =True))]),

    # ×欠番
    # AttributeError: 'LinearSVC' object has no attribute 'predict_proba'
    #
    #     # LinearSVC:inearSVCはカーネルが線形カーネルの場合に特化したSVMであり, 
    #     # 計算が高速だったり, 他のSVMにはないオプションが指定できたりする.
    #     'lsvc(LinearSVC)':
    #         Pipeline([('scl',StandardScaler()),
    #                   #('reduct', PCA(n_components=i_feature_num_val_adj, random_state=RANDOM_STATE_VAL)),
    #                   ('est',LinearSVC(C=1.0,class_weight='balanced',random_state=RANDOM_STATE_VAL))]),

#         # :ランダムフォレスト(精度が低いので封印)
#         ALG_NAME_RANDOMFOREST:
#             Pipeline([('scl',StandardScaler()),
#                       ('reduct', PCA(n_components=i_feature_num_val_adj, random_state=RANDOM_STATE_VAL)),
#                       ('est',RandomForestClassifier(random_state=RANDOM_STATE_VAL))]),
# 欠番
#             # :勾配ブースティング
#             'gb(勾配ブースティング)':
#                 Pipeline([('scl',StandardScaler()),
#                           ('reduct', PCA(n_components=i_feature_num_val_adj, random_state=RANDOM_STATE_VAL)),
#                           ('est',GradientBoostingClassifier(max_depth=10,random_state=RANDOM_STATE_VAL))]),

        # 2:多層パーセプトロン（MLP）
        ALG_NAME_MLP:
            Pipeline([('scl',StandardScaler()),
                      # TODO 必要に応じて次元圧縮復活
                      #('reduct', PCA(n_components=i_feature_num_val_adj, random_state=RANDOM_STATE_VAL)),
                      # デフォルト
                      # TODO warningが出たらmax_iterの値を増やすこと！！
                      ('est',MLPClassifier(max_iter=600,
                                           hidden_layer_sizes=(100,30,10),
                                           random_state=RANDOM_STATE_VAL))]),
        # 3:XGBoost
        ALG_NAME_XGB:
#            Pipeline([('scl',StandardScaler()),
            Pipeline([
                      # TODO 必要に応じて次元圧縮復活
                      #('reduct', PCA(n_components=i_feature_num_val_adj, random_state=RANDOM_STATE_VAL)),
                      # xg_tune_TPOT_mod.xml
                      ('est',xgb.XGBClassifier(max_depth=5,random_state=RANDOM_STATE_VAL)
                      # デフォルト
                      #('est',xgb.XGBClassifier(max_depth=10,random_state=RANDOM_STATE_VAL)
                     )])
    #　ハイパーパラメータ調整
    #                   ('est',GridSearchCV(estimator = xgb.XGBClassifier(
    #                                             random_state=RANDOM_STATE_VAL),
    #                         param_grid = xg_param1,n_jobs=-1,scoring=perf_label,cv=cv_num)  #cvのデフォルトは3
    #                   )])
    # 【重要】 デフォルトのパラメータ
    #                   ('est',xgb.XGBClassifier(max_depth=10,
    #                                            random_state=RANDOM_STATE_VAL)
    #                  )])

    }
    return pipelines
    
    
####################################
##### ↓精度の描画を行う関数
####################################
def plotPerf(rankDict, evalNumber):
    if(evalNumber == K_FOLD_NUMBER):
        # k-foldの場合
        colorlist=['red']
        titleHeader = "k-fold :  "
    else:
        # HoldOutの場合
        colorlist=['red','blue']
        titleHeader = "HoldOut :  "
    
    loopIndex = 0
    for key,valueDf in rankDict.items():
        # カラム「アルゴリズム」から右側だけ切り出す
        pltDf = valueDf.iloc[:,1:]
        
        # ホールドアウトの場合
        if(evalNumber == H_OUT_NUMBER):
            # 列名変更
            pltDf = pltDf.rename(columns={pltDf.columns.values[2]: 'train', pltDf.columns.values[1]: 'test'})

        # アルゴリズムをindexに指定
        pltDf = pltDf.set_index(pltDf.columns.values[0])
        # 描画
        pltDf.plot.bar(color=colorlist)
        plt.title(titleHeader + key, fontsize=PLT_TITLE_FONT_SIZE)
        plt.ylabel('精度', fontsize=PLT_LABEL_FONT_SIZE)
        plt.xlabel('アルゴリズム', fontsize=PLT_LABEL_FONT_SIZE)
        minVal = pltDf[pltDf.columns.values[0]].min()
        plt.ylim([(minVal - 0.05) , 1.0])   # y軸の最小メモリは最小値よりも0.05低い値
        plt.legend(bbox_to_anchor=(1.1, 1), loc='upper left')   #凡例を左下
        plt.show()


####################################
##### ↓k-foldの結果を返す関数
####################################
def getKFoldResult(perf_label, pipelines, X_input, y_input_ravel, i_cv_num, gridSearchFlag):
    print("=============================== getKFoldResult START ===============================")
    ret_k_fold_Df = pd.DataFrame(None,\
                     columns=['精度'])
    ##### ↓k-foldのforループ↓ #####
    pipe_index = 0
    for pipe_name, pipeline in pipelines.items():
        pipe_index = pipe_index + 1
        print("===" + pipe_name + "===")
        # 学習させる
        pipeline.fit(X_input, y_input_ravel)
# TODO cross_validateは、cross_val_scoreよりも便利らしいが、いったんここはcross_val_scoreで。
        cv_results = cross_val_score(pipeline,
                         X_input,
                         y_input_ravel,
                         cv=i_cv_num,
                         scoring=perf_label)
        ret_k_fold_Df.loc[pipe_name] = cv_results.mean()

        # TODO warning消したい。
        #C:\Users\USER\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.  % self.max_iter, ConvergenceWarning)
        #C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.  if diff:

####'fit_time',score_time','test_score', 'train_score'
# TODO cross_validateは、cross_val_scoreよりも便利らしいが、いったんここはcross_val_scoreで。
#         cv_results = cross_validate(pipeline,
#                          X_input,
#                          y_input_ravel,
#                          cv=cv_num,
#                          scoring=perf_label)
#         ret_k_fold_Df.loc[pipe_name] = cv_results['test_score'].mean()
#         print("平均学習時間:", cv_results['fit_time'].mean())
#         print("平均評価時間:", cv_results['score_time'].mean())
        
        if(gridSearchFlag == True):
            print("best_params：",pipeline.best_params_)
    ##### ↑k-foldのforループ↑ #####
    
    
    return ret_k_fold_Df


####################################
##### ↓HoldOutの結果を返す関数
####################################
def getHoldOutResult(perf_number, perf_label, pipelines, X_train, y_train_ravel, X_test, y_test_ravel):
    
    # fit & evaluation
    score_item_train = {}
    score_item_test = {}

    ##### ↓HoldOutのforループ↓ #####
    pipe_index = 0
    for pipe_name, pipeline in pipelines.items():
        pipe_index = pipe_index + 1
        print("===" + pipe_name + "===")

        # 学習させる
        pipeline.fit(X_train, y_train_ravel)

        # 【accuracy_score(正解率)】
        if perf_number == ACCURACY_NUMBER:
            score_item_train[(pipe_name,'train_' + perf_label)] = accuracy_score(y_train_ravel, 
                                                            pipeline.predict(X_train))
            score_item_test[(pipe_name,'test_' + perf_label)] = accuracy_score(y_test_ravel, 
                                                            pipeline.predict(X_test))
        # 【precision_score(適合率)】
        elif perf_number == PRECISION_NUMBER:
            score_item_train[(pipe_name,'train_' + perf_label)] = precision_score(y_train_ravel, 
                                                            pipeline.predict(X_train))
            score_item_test[(pipe_name,'test_' + perf_label)] = precision_score(y_test_ravel, 
                                                            pipeline.predict(X_test))
        # 【recall_score(再現率)】
        elif perf_number == RECALL_NUMBER:
            score_item_train[(pipe_name,'train_' + perf_label)] = recall_score(y_train_ravel, 
                                                            pipeline.predict(X_train))
            score_item_test[(pipe_name,'test_' + perf_label)] = recall_score(y_test_ravel, 
                                                            pipeline.predict(X_test))
        # 【f1_score(F値)】
        elif perf_number == F1_NUMBER:
            score_item_train[(pipe_name,'train_' + perf_label)] = f1_score(y_train_ravel, 
                                                            pipeline.predict(X_train))
            #デフォルトでpos_labelは1
            score_item_test[(pipe_name,'test_' + perf_label)] = f1_score(y_test_ravel, 
                                                            pipeline.predict(X_test))
            #デフォルトでpos_labelは1
        # 【AUC】
        elif perf_number == AUC_NUMBER:
            score_item_train[(pipe_name,'train_' + perf_label)] = roc_auc_score(y_train_ravel, 
                                                            pipeline.predict_proba(X_train)[:,TARGET_PROB_CLASS_VALUE])
        # ↓こう書いても一緒
#                                                            pipeline.predict_proba(X_train)[:,[TARGET_PROB_CLASS_VALUE]])
            score_item_test[(pipe_name,'test_' + perf_label)] = roc_auc_score(y_test_ravel, 
                                                            pipeline.predict_proba(X_test)[:,TARGET_PROB_CLASS_VALUE])
        # ↓こう書いても一緒
#                                                             pipeline.predict_proba(X_test)[:,[TARGET_PROB_CLASS_VALUE]])
        # あり得ないルート
        else:
            raise ValueError ("【ERROR】性能評価指標の値が不正です。")
    ##### ↑HoldOutのforループ↑ #####

    
    ###### ランキング出力
    ## unstack ： 行から列へのピボット 
    ##            インデックスが階層化されている場合にかぎり有効です。
    ##          Series→DataFrameになる。
    # TODO
    # warning消したい(f1で発生する。AUCでは発生しない。)
    #C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.  if diff:
    retRankDf = (pd.Series(score_item_train).unstack()).join(pd.Series(score_item_test).unstack())

    return retRankDf

####################################
##### ↓順位を付与してソートしたDFを返す関数
####################################
def getRankDf(inputDf, sortColumnName, oldIndexChangeName, firstTimeFlag):
    retRankDf = inputDf
    # 精度の大きい順に順位付けしてソートし直す。
    retRankDf['順位'] = (inputDf.rank(method='min',ascending=False)[sortColumnName]).astype('int')
    retRankDf = retRankDf.sort_values('順位', ascending=True)
    
    retRankDf = retRankDf.reset_index(drop=not(firstTimeFlag))
    if(firstTimeFlag):
        # 初回の場合
        # 列名の旧indexをoldIndexChangeNameに変更
        retRankDf = retRankDf.rename(columns={'index': oldIndexChangeName})
        
    return retRankDf
        
####################################
##### ↓予測して結果を返す関数
# 引数 学習済みモデル,検証用データの特徴量、予測種別、予測確率index
####################################
def getPredictResultDf(sumi_clf_pipeline, inputTestFeatureDf, predictType, probIndex):
    print("=============================== getPredictResultDf START ===============================")
    print("【第1引数】sumi_clf_pipeline：", sumi_clf_pipeline)
    displayPrintDf(inputTestFeatureDf,"【第2引数】inputTestFeatureDfの先頭%d行：" ,None,PRINT_DF_LINENUM)
    print("【第3引数】predictType：", predictType)
    print("【第4引数】probIndex：", probIndex)
    
    if(predictType == PREDICTTYPE_LABEL):
        # クラスのラベル
        # <class 'numpy.ndarray'>の1次元配列
        retArray = sumi_clf_pipeline.predict(inputTestFeatureDf)
        #colName = 'Class'
        colName = 'Survived'
    else:
        # 予測確率
        # <class 'numpy.ndarray'>の2次元配列
        retArray = sumi_clf_pipeline.predict_proba(inputTestFeatureDf)[:,probIndex]
        colName = conf_col_proba
        
    # DataFrameに変換
    retDf = pd.DataFrame(data=retArray, columns=[colName])
    displayPrintDf(retDf,"予測結果の先頭%d行：" ,None,PRINT_DF_LINENUM)
    return retDf




●メイン処理++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
#gray
def main():
    # ●メイン処理-データ取得・加工処理 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    ################################################################################
    # 全般的にfor文はlambda式で書き直せるものはそうしたいところだが、
    # 時間的にアレなので、余裕があればやる。
    ################################################################################
    global feature_num_lim_val


    # cross validation(10固定にしようと思ったが、時間がかかりすぎる可能性があるのでやめた。)
    ###cv_num =  int(getSettingValue(conf_reha_cv_num, "10", conf_exe_mode))
    cv_num =  conf_cv_num

    print("cv_num = %d" % cv_num)

    # pklファイル名リスト取得
    pkl_file_list=os.listdir(conf_pkl_folder_path)
    print("pkl_file_list = ", pkl_file_list)


    # pkl自動削除モードONの場合
    if(conf_pkl_del_mode == FLAG_ON):
        print("pkl自動削除モード=ON")
        #pklファイルを削除する。
        for pkl_file in pkl_file_list:
            os.remove(conf_pkl_folder_path + pkl_file)
        # 自動削除後に再度pklファイル名リスト取得
        pkl_file_list=os.listdir(conf_pkl_folder_path)


    if(len(pkl_file_list) > 0):
        print("************************************* pklあり(2回目以降) *************************************")
        print("== pklファイルから読み込む ==")
        with open(conf_pkl_folder_path + 'trainFeatureDf.pkl', 'rb') as DFfile:
            trainFeatureDf = pickle.load(DFfile)
        with open(conf_pkl_folder_path + 'testFeatureDf.pkl', 'rb') as DFfile:
            testFeatureDf = pickle.load(DFfile)
        with open(conf_pkl_folder_path + 'y_train.pkl', 'rb') as DFfile:
            y_train = pickle.load(DFfile)
        with open(conf_pkl_folder_path + 'idDf.pkl', 'rb') as DFfile:
            idDf = pickle.load(DFfile)
    else:
        print("************************************* pklなし(初回実行) *************************************")

        # 特殊カラム対象の決定 ########################################################
        #### oneHotEncoderExe対象のカラム名を取得してohe_clm_listに保持する。
        ohe_clm_list = getFileValueList(conf_one_hot_encode_column_sitei, conf_input_folder_path,\
            getSettingValue(conf_reha_one_hot_encode_column, "OneHotEncoderColumns.dat", conf_exe_mode))

        #### 年月日時分秒対象のカラム名を取得してdatetime_clm_listに保持する。
        datetime_clm_list = getFileValueList(conf_datetime_column_sitei, conf_input_folder_path,\
            getSettingValue(conf_reha_datetime_column, "DateTime.dat", conf_exe_mode))

        # 訓練用データcsvファイルを読み込みんでDfを返す。
        trainInputDf = getDfByReadCSV(FILE_TYPE_NAME_TRAIN, conf_exe_mode,\
            conf_reha_train_input_file , ohe_clm_list, datetime_clm_list, conf_datetime_column_sitei)

        # 検証用データcsvファイルを読み込みんでDfを返す。
        testInputDf = getDfByReadCSV(FILE_TYPE_NAME_TEST, conf_exe_mode,\
            conf_reha_test_input_file , ohe_clm_list, datetime_clm_list, conf_datetime_column_sitei)

        print("# ======================================================================================================")
        print("# ======================================================================================================")
        print("# ======================================================================================================")

        ######### 訓練用データ  ########################################################
        # 1列目(ID)と2列目(クラス変数)を削除することにより、第3カラム以降(特徴量)のみ残す
        trainFeatureDf = trainInputDf.drop(columns=[trainInputDf.columns.values[0], trainInputDf.columns.values[CLASS_COL_INDEX]],axis=1)

        # OneHotEncoder
        trainFeatureDf = oneHotEncoderExe(FILE_TYPE_NAME_TRAIN, ohe_clm_list, trainFeatureDf, conf_one_hot_encode_column_sitei)

        # 時刻を経過時間に変換
        ##trainFeatureDf = getConvTimeDf(FILE_TYPE_NAME_TRAIN, trainFeatureDf)

        # 欠損値を埋める(平均値で埋める)
        displayPrintDf(trainFeatureDf, "++++++ ↓%sの欠損埋め前先頭%d行↓ ++++++",FILE_TYPE_NAME_TRAIN,PRINT_DF_LINENUM)
        trainFeatureDf = trainFeatureDf.fillna(trainFeatureDf.mean())
        displayPrintDf(trainFeatureDf, "++++++ ↓%sの欠損埋め後先頭%d行↓ ++++++",FILE_TYPE_NAME_TRAIN,PRINT_DF_LINENUM)


        # # ガード処理。
        # # 以下のエラーが出るときがある。
        # # ValueError: Input contains NaN, infinity or a value too large for dtype('float64').
        # # これを避けるためには、入力データからNaNや無限大の列を除去する必要がある。
        # trainFeatureDf = trainFeatureDf.drop(trainFeatureDf.columns[np.isnan(trainFeatureDf).any()], axis=1)
        # displayPrintDf(trainFeatureDf,"ガード処理後の%sの先頭%d行：" ,FILE_TYPE_NAME_TRAIN,PRINT_DF_LINENUM)

        print("# ======================================================================================================")
        print("# ======================================================================================================")
        print("# ======================================================================================================")
        ######### 検証用データ  ########################################################

        # 1列目(ID)と2列目(クラス変数)を削除することにより、第3カラム以降(特徴量)のみ残す
        testFeatureDf = testInputDf.drop(columns=[testInputDf.columns.values[0], testInputDf.columns.values[CLASS_COL_INDEX]],axis=1)

        # OneHotEncoder
        testFeatureDf = oneHotEncoderExe(FILE_TYPE_NAME_TEST, ohe_clm_list, testFeatureDf, conf_one_hot_encode_column_sitei)

        # 時刻を経過時間に変換
        ##testFeatureDf = getConvTimeDf(FILE_TYPE_NAME_TEST, testFeatureDf)

        # 欠損値を埋める(平均値で埋める)
        displayPrintDf(testFeatureDf, "++++++ ↓%sの欠損埋め前先頭%d行↓ ++++++",FILE_TYPE_NAME_TEST,PRINT_DF_LINENUM)
        testFeatureDf = testFeatureDf.fillna(testFeatureDf.mean())
        displayPrintDf(testFeatureDf, "++++++ ↓%sの欠損埋め後先頭%d行↓ ++++++",FILE_TYPE_NAME_TEST,PRINT_DF_LINENUM)


        # # ガード処理。
        # # 標準化の際に以下のエラーが出るときがある。
        # # ValueError: Input contains NaN, infinity or a value too large for dtype('float64').
        # # これを避けるためには、入力データからNaNや無限大の列を除去する必要がある。
        # testFeatureDf = testFeatureDf.drop(testFeatureDf.columns[np.isnan(testFeatureDf).any()], axis=1)
        # displayPrintDf(testFeatureDf,"ガード処理後の%sの先頭%d行：" ,FILE_TYPE_NAME_TEST,PRINT_DF_LINENUM)

        ###############################################################
        # 正解データ
        y_train = trainInputDf.iloc[:,[CLASS_COL_INDEX]]

        # 試験用データのIDのカラムを取り出してjoinに備える。
        idDf = testInputDf.iloc[:,[0]]

        # 不均衡調整(モデルに課題がなければやる必要はない)
        # バグってる気がするので封印
        # trainFeatureDf, y_train = unbalanceAdj(conf_unbalance_adj_mode, trainFeatureDf, y_train)

        print("== pklファイルに残す ==")
        with open(conf_pkl_folder_path + 'trainFeatureDf.pkl', 'wb') as DFfile:
            pickle.dump(trainFeatureDf, DFfile, protocol=2)
        with open(conf_pkl_folder_path + 'testFeatureDf.pkl', 'wb') as DFfile:
            pickle.dump(testFeatureDf, DFfile, protocol=2)
        with open(conf_pkl_folder_path + 'y_train.pkl', 'wb') as DFfile:
            pickle.dump(y_train, DFfile, protocol=2)
        with open(conf_pkl_folder_path + 'idDf.pkl', 'wb') as DFfile:
            pickle.dump(idDf, DFfile, protocol=2)


    ############################################################################
    ############################# ↓メインループ↓ #############################
    ############################################################################
    continueFlag = True
    feature_num_list = list()
    while continueFlag:
        print("*****************************************************************************************************************")
        print("feature_num_lim_val = %d : START" % feature_num_lim_val)
        print("*****************************************************************************************************************")

        #####################
        # 選択後の訓練用データのpklファイルパス
        train_s_pkl_file = conf_pkl_folder_path + 'trainFeatureDf_s_' + str(feature_num_lim_val) + '.pkl'

        # 選択後の訓練用データのpklファイルの有無判定
        if(os.path.isfile(train_s_pkl_file)):
            # 選択後の訓練用データのpklあり
            print("++ " + train_s_pkl_file + " あり ++")

            # 訓練用データの特徴量をロード
            with open(train_s_pkl_file, 'rb') as DFfile:
                trainFeatureDf_s = pickle.load(DFfile)
        else:
            # 選択後の訓練用データのpklなし
            print("++ " + train_s_pkl_file + " なし ++")

            # 特徴量選択
            trainFeatureDf_s = getSelectorFeature(FILE_TYPE_NAME_TRAIN, trainFeatureDf, \
                 y_train, feature_num_lim_val)

            # 訓練用データの特徴量をダンプ(HyperOptExe用にダンプする)
            with open(train_s_pkl_file, 'wb') as DFfile:
                pickle.dump(trainFeatureDf_s, DFfile, protocol=2)

        #####################
        # 選択後の検証用データのpklファイルパス
        test_s_pkl_file = conf_pkl_folder_path + 'testFeatureDf_s_' + str(feature_num_lim_val) + '.pkl'

        # 選択後の検証用データのpklファイルの有無判定
        if(os.path.isfile(test_s_pkl_file)):
            # 選択後の検証用データのpklあり
            print("++ " + test_s_pkl_file + " あり ++")

            # 検証用データの特徴量をロード
            with open(test_s_pkl_file, 'rb') as DFfile:
                testFeatureDf_s = pickle.load(DFfile)
        else:
            # 選択後の検証用データのpklなし
            print("++ " + test_s_pkl_file + " なし ++")

            # 検証用のカラムを、訓練用に存在するやつだけ残す。無ければ列をデフォルト値で追加する。
            # カラム順の並び替えも行う。
            testFeatureDf_s = adjustTestDfColumn(trainFeatureDf_s, testFeatureDf)

            # 検証用データの特徴量をダンプ(HyperOptExe用にダンプする)
            with open(test_s_pkl_file, 'wb') as DFfile:
                pickle.dump(testFeatureDf_s, DFfile, protocol=2)

        #####################

        # feature_特徴量上限数_事前評価方法 ディレクトリ
        feature_dir = "feature_" + str(feature_num_lim_val) + "_" + evalDict.get(conf_eval_method) + "/"

        # モデル選択
        retClfPipe = getBestModelClf(trainFeatureDf_s, y_train, feature_num_lim_val, feature_dir, cv_num)

        # ●メイン処理-予測 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        # 予測
        # 引数 学習済みモデル、検証用データの特徴量、予測種別、 クラスINDEX
        resultDf = getPredictResultDf(retClfPipe, testFeatureDf_s, PREDICTTYPE_LABEL, TARGET_PROB_CLASS_VALUE)  #予測確率(クラス1)

        # 不均衡調整モードを有効にしない限り必要の無い処理
        ##idDf = idDf.reset_index(drop=True)

        # IDと予測結果をjoin
        resultDf = idDf.join(resultDf)

        # ●メイン処理-ファイル出力 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

        # 選択されたモデルをファイルに残す。
        if(type(retClfPipe) is GridSearchCV):
            selectedModel = str(retClfPipe)
        else:
            selectedModel = str(retClfPipe.named_steps['est'])
        print(selectedModel)
        modelFilePath =  conf_output_folder_path + feature_dir + 'selected_model.txt'
        with open(modelFilePath, mode='w') as f:
            f.write(selectedModel)

        #################

        # 予測結果CSVファイル出力
        fileOutput(resultDf, "", conf_submission_file, PRINT_DF_LINENUM, True, feature_dir)

        # 実施リストに追加
        feature_num_list.append(feature_num_lim_val)

        feature_num_lim_val = getInputNumber("特徴量数(半角整数)", 
                       "\r\nこれで終了する場合は空文字のままEnterを押してください。 \
                        \r\nこれまでに予測した特徴量：" + str(feature_num_list) + "")

        if(feature_num_lim_val == None):
            # ループ脱出
            continueFlag = False
    ############################################################################
    ############################# ↑メインループ↑ #############################
    ############################################################################

    #終了
    print("++++++++++++++++++++++++ END ++++++++++++++++++++++++")


#####################################################################
#####################################################################
# メイン処理実行
#####################################################################
#####################################################################
if __name__ == '__main__':
    main()